In [ ]:
import os, fileinput, random, datetime
import re, json, csv
import itertools, collections
import numpy, pandas, math, scipy, scipy.stats
import plotly
import climata

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import climata.usgs as usgs

from functools import reduce
from collections import namedtuple
from math import sqrt
from IPython.core.display import display, HTML
from numpy import nan
from pprint import pprint, pformat

In [ ]:
# dumb setup stuff\n
display(HTML("<style>.container { width:100% !important; }</style>"))
# matplotlib.rcParams['figure.figsize'] = [8, 5
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,f}'.format

my_template = dict(
  
)

pio.templates["ians_template"] = go.layout.Template(
  layout=go.Layout(
    height=700
  )
)
pio.templates.default = "ians_template"

In [ ]:
def merge_dictionary_lists( *dictionaries ):
  # Simply use keys from first dictionary
  # Ensure later that all dictionaries have these keys
  keys = set( dictionaries[0].keys() )
  
  if len(dictionaries) == 1: 
    # Check keys are the same by comparing to first
    for d in dictionaries[1:]:
      if set(d.keys()) != keys:
        raise ValueError( "All dictionaries in merge_dictionary_lists must have same keys" )
    
  result_dict = { 
    key : list( itertools.chain( *( list(d[key]) for d in dictionaries ) ) )
    for key in keys 
  }
  
  return result_dict
  

def station_id_cast( station_id ):
  if isinstance( station_id, int ):
    return f"{station_id:0>8}"
  if isinstance( station_id, str ):
    return station_id_cast( int(station_id, 10) )
  if isinstance( station_id, float ):
    # check is int-able
    if int(math.ceil(station_id)) != int(math.floor(station_id)):
      raise ValueError( f"Cannot create station id from non-integer float {station_id}" )
    return station_id_cast( int(station_id) )
  
  raise ValueError( f"station_id_cast given station id as {type(station_id)}, can only handle int, str, and float" )
  

def parameter_id_cast( parameter_id ):
  if isinstance( parameter_id, int ):
    return f"{parameter_id:0>5}"
  if isinstance( parameter_id, str ):
    return parameter_id_cast( int(parameter_id, 10) )
  if isinstance( parameter_id, float ):
    # check is int-able
    if int(math.ceil(parameter_id)) != int(math.floor(parameter_id)):
      raise ValueError( f"Cannot create parameter id from non-integer float {parameter_id}" )
    return parameter_id_cast( int(parameter_id) )
  
  raise ValueError( f"parameter_id_cast given parameter id as {type(parameter_id)}, can only handle ints, strings, and floats" )

  
def load_station_data_over_date_range_as_DataFrame( station_id, parameter_id, start_date=None, end_date=None, periods=None, date_range=None ):
  # Ensure correct invocation of function
  if date_range is None :
    # for pd.date_range, must define exactly 3 of the 4 time-period parameters (in our case, at least 2 of 3)
    # Done by counting how many of the parameters are None
    if len( [ i for i in [start_date, end_date, periods] if i is not None] ) < 2:
      raise ValueError( "With date_range unset, at least 2 of the following parameters must be specified: start_date, end_date, and periods" )
  
  # Properly cast station and parameter ids
  station_id = station_id_cast( station_id )
  parameter_id = parameter_id_cast( parameter_id )
                       
  # Create date-range 
  # From provided time-period parameters
  if date_range is None:
    date_range_list = pd.date_range( start=start_date, end=end_date, periods=periods ).tolist()
  # From provided list of dates
  elif isinstance( date_range, list ):
    date_range_list = date_range
  # From provided pandas DatetimeIndex
  elif isinstance( date_range, pandas.core.indexes.datetimes.DatetimeIndex ):
    date_range_list = date_range.tolist()

  # Download station data
  station_requests = usgs.DailyValueIO(
      start_date = date_range_list[0],
      end_date   = date_range_list[-1],
      station    = station_id,
      parameter  = parameter_id
    )
  # climata.usgs.DailyValueIO claims to be iterable, but in practice there is nothing to iterate over...
  # Note the IO obeject annot be exhausted
  #(ie, it can be iterated over multiple times to get the different aspects of the request(s) from it)
  
  # Flatten data into list of dictionaries with the data and flow extracted
  # Note this is a generator, which *CAN* be exhausted. Only use *ONCE*!
  flattened_data_generator = (
    dict(
      date=date_value_object.date,
      day=date_value_object.date.timetuple()[7]-1,       # Days since January 1st
      week=int(math.floor((date_value_object.date.timetuple()[7]-1) / 7)), # Week of the year
      month=date_value_object.date.month,
      year=date_value_object.date.year,
      flow=date_value_object.value
    )
    for request in station_requests
    for date_value_object in request.data
  )
  
  # Create DatFrame from the flattened generator.
  station_data = pd.DataFrame( 
    columns = ["date","day","week","month","year","flow"],
    data    = flattened_data_generator 
  ).sort_values("date")
  
  return station_data


def load_station_mapping_as_DataFrame( path_to_stations_table ):
  return pd.read_csv( path_to_stations_table )


def evaluate_parflow( station_id, parameter_id, stations_map, parflow_data, start_date=None, end_date=None, periods=None, date_range=None, group_by=None ):
  station_id = station_id_cast( station_id )
  
  # Load Station Data
  station_data = load_station_data_over_date_range_as_DataFrame( station_id=station_id, parameter_id=parameter_id, start_date=start_date, end_date=end_date, periods=periods, date_range=date_range )
  
  # Get this station's ParFlow x and y mapping
  station_x, station_y = stations_map[ stations_map["STNID"] == int(station_id) ][["x_new","y_new"]].values[0]
  
  # Filter ParFlow data down to this station.
  parflow_station_data = parflow_data[ (parflow_data["x"] == station_x) & (parflow_data["y"] == station_y) ].sort_values("date")
  
  # Group (as necessary)
  if group_by != None:
    valid_group_by_keys = ["week", "month", "year"]
    if group_by not in valid_group_by_keys:
      raise RuntimeError( f"Cannot group by \"{group_by}\". Can only group by {', '.join( valid_group_by_keys )}" )
    
    grouped_station_data = station_data.groupby( group_by, as_index=False ).aggregate( {"date" : "first", "flow" : "mean"} )
    grouped_parflow_station_data = parflow_station_data.groupby( group_by, as_index=False ).aggregate( {"date" : "first", "flow" : "mean"} )
    yaxis_title_modifier = f"Average {group_by.title()}ly"                                                                                        
  else:
    grouped_station_data = station_data
    grouped_parflow_station_data = parflow_station_data
    yaxis_title_modifier = "Daily"
    
#   print( grouped_station_data )
#   print( grouped_parflow_station_data )

  # Compute differences
  parflow_station_difference = pd.DataFrame( 
    dict(
      date = grouped_station_data["date"],
      flow = grouped_parflow_station_data["flow"] - grouped_station_data["flow"]
    )
  ).sort_values("date")
  
  parflow_station_difference_squared = pd.DataFrame( 
    dict(
      date = grouped_station_data["date"],
      flow = parflow_station_difference["flow"].pow(2)
    )
  ).sort_values("date")
    
  
  N = grouped_station_data.index.size
  parflow_station_difference_sum = parflow_station_difference["flow"].sum()
  parflow_station_difference_squared_sum = parflow_station_difference_squared["flow"].sum()

  percent_bias = ( parflow_station_difference_sum / grouped_station_data["flow"].sum() )*100
  # For scipy.stats.spearmanr, I am not under the impression that the order matters
  spearman_rho, spearman_rho_p = scipy.stats.spearmanr( grouped_parflow_station_data["flow"], grouped_station_data["flow"] )
  r_squared = 1 - ( parflow_station_difference_squared_sum / (grouped_station_data["flow"] - grouped_station_data["flow"].mean()).pow(2).sum() )
  rmse = np.sqrt( parflow_station_difference_squared_sum / N )
  
  print( f"% Bias: {percent_bias}%" )
  print( f"Spearman Rho: {spearman_rho} (p={spearman_rho_p})" )
  print( f"R2: {r_squared}" )
  print( f"RMSE: {rmse}" )

  go.Figure(
    data=[
      go.Scatter( 
        x=grouped_station_data["date"],
        y=grouped_station_data["flow"],
        name="Actual",
        mode="lines+markers",
        line=dict(
          color="green"
        )
      ),
      go.Scatter( 
        x=grouped_parflow_station_data["date"],
        y=grouped_parflow_station_data["flow"],
        name="Parflow",
        mode="lines+markers",
        line=dict(
          color="orange"
        )
      ),
      go.Scatter( 
        x=parflow_station_difference["date"],
        y=parflow_station_difference["flow"],
        name="True Difference (ParFlow - Station)",
        mode="lines+markers",
        line=dict(
          color="red"
        )
      )
    ],
    layout=dict(
      title=f"Comparison of Flow between Station {station_id} and ParFlow @ ({station_x}, {station_y})",
      xaxis=dict(
        title="Date"
      ),
      yaxis=dict(
        title=f"{yaxis_title_modifier} Flow"
      )

    )
  ).show()

In [ ]:
# Stations map
stations_map_path = "NWM_Gage_Adjustments_attribute_table_20200510.csv"
station_map = load_station_mapping_as_DataFrame( stations_map_path )

# Station info
station_id = station_id_cast( "01011000" )
parameter_id = parameter_id_cast( "00060" )
# data-range
end_date="2010-12-31"
periods=365
date_range=pd.date_range(end=end_date, periods=periods).tolist()

# Fake some parflow data starting from the actual station date
# I assume the data will look something like a table of rows in the form:
# x, y, Date, Parameter Value
# Here, use few stations the station (x,y,z) associated with station id.
parflow_data = pd.DataFrame(
  columns = ["x", "y", "date","day","week","month","year", "flow"],
  data    = merge_dictionary_lists(
    *(
      {
        "x" : [ station_map[ station_map["STNID"] == station_id_int ]["x_new"].values[0] ] * len(actual_data),
        "y" : [ station_map[ station_map["STNID"] == station_id_int ]["y_new"].values[0] ] * len(actual_data),
        "date" : actual_data["date"],
        "day" : actual_data["day"],
        "week" : actual_data["week"],
        "month" : actual_data["month"],
        "year" : actual_data["year"],
        "flow" : actual_data["flow"] * (random.random() + random.random())  + (np.random.randn(len(date_range)) * np.random.randn(len(date_range)))
      }
      for station_id_int in [ int( station_id ), 1034000, 1068910 ]
      # Hacky let expression
      for actual_data in [load_station_data_over_date_range_as_DataFrame( station_id_cast(station_id_int), parameter_id, date_range=date_range)]
    )
  )
)
# parflow_data[0:366]
# print( load_station_data_over_date_range_as_DataFrame( station_id, parameter_id, date_range=date_range ) )

In [ ]:
evaluate_parflow( 
  station_id,
  parameter_id,
  station_map,
  parflow_data,
  date_range=date_range,
)
evaluate_parflow( 
  station_id,
  parameter_id,
  station_map,
  parflow_data,
  date_range=date_range,
  group_by="week"
)
evaluate_parflow( 
  station_id,
  parameter_id,
  station_map,
  parflow_data,
  date_range=date_range,
  group_by="month"
)